In [ ]:
"""
Quick visualization of data along a ICESAT2 Track
To understand more about the data being input to this script, consider visiting https://nsidc.org/sites/default/files/atl10-v005-userguide_1_0.pdf
Code written originally by: Chris Polashenski
Refactored, added command line support, and removed sartopy package: John Baker
"""

import h5py
import pandas as pd
import datetime
import numpy as np
import sys
import haversine as hs


In [ ]:
in_file = "D:\\ICEEYE\\Proposal_ID_PP0091278\\CS-13801\\SLEA_3279211_180312\\Baker\\Coincident Data\\ATL10QL-01_20240119042539_04792201_006_01.h5"
print(in_file)
path_parts = in_file.split("\\")
file_name = path_parts[-1]
del path_parts[-1]

path_parts.append(file_name[0:-3])
out_file = '/'.join(path_parts) + ".csv"
print(out_file)

In [ ]:
# Consider adding a for loop to do this same process but for all 6 beams gt1r/l, gt2r/l, gt3r/l
# Ex: Declare the starting beam names, and iterate through that array and push the variables into another array
#       where you then supply them into df
with h5py.File(in_file, mode='r') as file:
    print(list(file.keys()))
    # We'll need to restructure our script to call functions that pull this information out.
    # We could make a function and pass in these variables as keys? That way we can avoid drawing them out.
    beam_enum = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

    df_dict = {
        'Beam': [],
        "Lat": [],
        "Lon": [],
        "FBH": [],
        "Dist(m)": [],
        "RS#(1-based)": [],
        "RSDist": [],
        "RSType": [],
        "RsMSS": [],
        "RsHeight": [],
        "Calendar Time": []
    }

    if "gt1r/freeboard_beam_segment/beam_freeboard" in file:
        filePathToBeamData = "freeboard_beam_segment/beam_freeboard"
    else:
        filePathToBeamData = "freeboard_segment"

    if "gt1r/freeboard_beam_segment/reference_surface_section" in file:
        filePathToReferenceHeights = "freeboard_beam_segment/reference_surface_section"
    else:
        filePathToReferenceHeights = "reference_surface_section"

    print(filePathToBeamData)
    print(filePathToReferenceHeights)

    for beam in beam_enum:
        ############################### Store the reference surface data ########################################
        # Read in the distance from the reference surface
        refSurfDist = file[f'/{beam}/{filePathToReferenceHeights}/beam_refsurf_dist_x']
        refSurfDist = list(refSurfDist[:])

        # Read in the interpolation flag of the reference surface
        refSurfType = file[f"/{beam}/{filePathToReferenceHeights}/beam_refsurf_interp_flag"]
        refSurfType = list(refSurfType[:])

        # Read in mean sea surface height (the base elevation everything is referring to)
        mss = file[f"/{beam}/{filePathToReferenceHeights}/beam_refsurf_mss"]
        mss = list(mss[:])

        # Read in the inferred ice elevation (the base elevation everything is referring to)
        refSurfHeight = file[f"/{beam}/{filePathToReferenceHeights}/beam_refsurf_height"]
        refSurfHeight = list(refSurfHeight[:])
        #######################################################################      

        # Read in the lat/lon
        latvar = file[f'/{beam}/{filePathToBeamData}/latitude']
        latitude = list(latvar[:])
        lonvar = file[f'/{beam}/{filePathToBeamData}/longitude']
        longitude = list(lonvar[:])

        # Read in the freeboard height
        fb_data = f'/{beam}/{filePathToBeamData}/beam_fb_height'
        datavar = file[fb_data]
        data = datavar[:]

        # Assign the Fill Value once
        _FillValue = datavar.attrs['_FillValue']

        # Handle FillValue
        data[data == _FillValue] = np.nan
        data = np.ma.masked_where(np.isnan(data), data)

        # Read in the time and convert to readable
        timevar = file[f'/{beam}/{filePathToBeamData}/delta_time']
        time = list(timevar[:])
        dateTime = []
        GPS_EPOCH = datetime.datetime(2018,1,1).timestamp()    # Passing in the ATLAS EPOCH

        for timeInSec in time:
            calendarTime = datetime.datetime.fromtimestamp(timeInSec+GPS_EPOCH).strftime('\"%H:%M:%S.%f %Y-%m-%d\"')
            dateTime.append(calendarTime)

        # Read in the beam confidence
        dataConf = file[f'/{beam}/{filePathToBeamData}/beam_fb_confidence']
        dataConf = list(dataConf[:])

        # Read in the quality flag
        qualityFlag = file[f"/{beam}/{filePathToBeamData}/beam_fb_quality_flag"]
        qualityFlag = list(qualityFlag[:])

        # Read in the ref height index
        refSurfIdxs = file[f'/{beam}/{filePathToBeamData}/beam_refsurf_ndx']
        refSurfIdxs = list(refSurfIdxs[:])

        refSurfDists = []
        refSurfTypes = []
        refSurfMss = []
        refSurfHeights = []
        for i in range(len(refSurfIdxs)):
            pointRefSurfDist = refSurfDist[refSurfIdxs[i]-1]
            pointRefSurfType = refSurfType[refSurfIdxs[i]-1]
            pointMss = mss[refSurfIdxs[i]-1]
            pointRefSurfHeight = refSurfHeight[refSurfIdxs[i]-1]

            refSurfDists.append(pointRefSurfDist)
            refSurfTypes.append(pointRefSurfType)
            refSurfMss.append(pointMss)
            refSurfHeights.append(pointRefSurfHeight)

        print("Max refSurf Idx: ", max(refSurfIdxs))
        print("Min refSurf Idx: ", min(refSurfIdxs))

        # Calculate ground distance
        distance = []
        x1 = longitude[0]
        y1 = latitude[0]

        for i in range(len(latitude)):
            x2 = longitude[i]
            y2 = latitude[i]
            deltaDistance = hs.haversine((y1,x1), (y2, x2), normalize=True, unit="m")
            distance.append(deltaDistance)
            x1= x2
            y1 = y2
        # freeboard_segment/beam_fb_length
        #   reference_surface_section/beam_refsurf_dist_x
        #   reference_surface_section/beam_refsurf_interp_flag
        #   reference_surface_section/beam_refsurf_mss
        #   freeboard_segment/beam_refsurf_height

        print("Beam: ", beam)
        print("Freeboards & location lengths")
        print(len(latitude))
        print(len(longitude))
        print(len(data))

        print("Confidence lengths:")
        print(len(dataConf))

        print("Ref Surf Data")
        print(len(refSurfIdxs))
        print(len(refSurfDists))
        print(len(refSurfTypes))
        print(len(refSurfMss))
        print(len(refSurfHeights))

        print("Time")
        print(len(time))
        print(len(dateTime))

        # We want to append the lat/lon, fbheights, ground distance, refSurfance data for each beam, and then dateTime only on the last
        latCol = beam + "Lat"
        lonCol = beam + "Lon"
        fbCol = beam + "FBH"
        distCol = beam + "Dist(m)"
        rsIndex = beam + "RS#(1-based)"
        rsDist = beam + "RSDist"
        rsType = beam + "RSType"
        rsMss = beam + "RsMSS"
        rsHeight = beam + "RsHeight"

        for i in range(len(latitude)):
            df_dict['Beam'].append(beam)
            df_dict["Lat"].append(latitude[i])
            df_dict["Lon"].append(longitude[i])
            df_dict["FBH"].append(data[i])
            df_dict["Dist(m)"].append(distance[i])
            df_dict["RS#(1-based)"].append(refSurfIdxs[i])
            df_dict["RSDist"].append(refSurfDists[i])
            df_dict["RSType"].append(refSurfTypes[i])
            df_dict["RsMSS"].append(refSurfMss[i])
            df_dict["RsHeight"].append(refSurfHeights[i])
            df_dict["Calendar Time"].append(dateTime[i])


In [ ]:
                
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in df_dict.items()]))
df = df.replace(r'^\s*$', np.nan, regex=True)

print(df.head(5))
print(df.shape)
# Drop all rows where ICESAT has no FB height data
val = df.iloc[0]['FBH']
filtered = df[ df['FBH'] != val ]
print(filtered.head(5))
print(filtered.shape)


filtered.to_csv(out_file, date_format=None)

print("\nSuccesfully wrote to output csv. Dataframe shape:")
print("\n")